In [12]:
import os
import sys
sys.path.append(os.path.join('..', 'src'))
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from genes_annotations import *
from autoencoder import *
from utils import *

In [2]:
filename = "../data_tisch2/Glioma_GSE102130_expression.h5"

h5 = read_h5(filename)

selected_genes = get_all_genes(h5)

In [3]:
genes_go, all_go, all_genes = get_all_gene_annotations(h5)

HMS:0:00:08.412569 606,840 annotations READ: ../goa_human.gaf 
../go.obo: fmt(1.2) rel(2021-02-01) 50,515 GO Terms; optional_attrs(consider replaced_by)


In [4]:
mask = build_mask(h5,genes_go,all_go,all_genes)

In [5]:
expr_mat = build_expr_mat(h5)
expr_mat = select_gene_expr_v2(h5, expr_mat, all_genes)

'build_expr_mat'  442.62 ms
'select_gene_expr_v2'  362.61 ms


### Récupérer les labels

In [6]:
df = pd.read_csv('../data_tisch2/Glioma_GSE102130_CellMetainfo_table.tsv', delimiter='\t')
targets = np.array(df['Cluster'])
N_classes = targets.max()+1

In [7]:
np.sum(targets==1)

488

### Entrainement sur le groupe 0

In [8]:
dataset0 = expr_mat[targets==0, :]

In [18]:
ae0, train_loader0, embeddings0 = ae_pipeline(mask, dataset0, n_epochs=50, batch_size=5, print_loss=100, output_file='model_0.pth', embed_file='embeddings_ae_0.csv')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              Tanh-1              [-1, 1, 1414]               0
      MaskedLinear-2              [-1, 1, 1414]               0
            Linear-3               [-1, 1, 500]         707,500
              Tanh-4               [-1, 1, 500]               0
            Linear-5              [-1, 1, 1414]         708,414
              Tanh-6              [-1, 1, 1414]               0
              Tanh-7             [-1, 1, 10232]               0
      MaskedLinear-8             [-1, 1, 10232]               0
Total params: 1,415,914
Trainable params: 1,415,914
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 0.21
Params size (MB): 5.40
Estimated Total Size (MB): 5.65
----------------------------------------------------------------
[1,     1] loss: 0.90373
[1,   101] lo

### Entrainement sur le groupe 1

In [14]:
dataset1= expr_mat[targets==1, :]

In [16]:
ae, train_loader, embeddings = ae_pipeline(mask, dataset1, n_epochs=50, batch_size=5, print_loss=100, output_file='model_1.pth', embed_file='embeddings_ae_1.csv')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
              Tanh-1              [-1, 1, 1414]               0
      MaskedLinear-2              [-1, 1, 1414]               0
            Linear-3               [-1, 1, 500]         707,500
              Tanh-4               [-1, 1, 500]               0
            Linear-5              [-1, 1, 1414]         708,414
              Tanh-6              [-1, 1, 1414]               0
              Tanh-7             [-1, 1, 10232]               0
      MaskedLinear-8             [-1, 1, 10232]               0
Total params: 1,415,914
Trainable params: 1,415,914
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.04
Forward/backward pass size (MB): 0.21
Params size (MB): 5.40
Estimated Total Size (MB): 5.65
----------------------------------------------------------------
[1,     1] loss: 0.99211
[2,     1] lo

In [17]:
ae1, train_loader1, embeddings1 = ae, train_loader, embeddings

In [19]:
ae0

GeneAutoEncoder(
  (encoder): Sequential(
    (0): MaskedLinear(
      (activation): Tanh()
    )
    (1): Linear(in_features=1414, out_features=500, bias=True)
    (2): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=500, out_features=1414, bias=True)
    (1): Tanh()
    (2): MaskedLinear(
      (activation): Tanh()
    )
  )
)